In [1]:
with open('input.txt' , 'r', encoding = 'utf-8') as f:
        text = f.read()

In [2]:
len(text)

1115394

In [3]:
text[:10]

'First Citi'

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


In [5]:
vocab_size

65

In [6]:
chars

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [7]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [8]:
a = encode("hi")
a

[46, 47]

In [9]:
print(decode(a))

hi


In [10]:
import torch
data = torch.tensor(encode(text),dtype = torch.long)

In [11]:
print(data.shape)

torch.Size([1115394])


In [12]:
data.dtype

torch.int64

In [13]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [14]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data [n:]

In [15]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [39]:
torch.manual_seed(1337)
batch_size = 4
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [40]:
 xb,yb = get_batch('train')

In [41]:
xb,yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [42]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [ ]:
class BigramModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,inputs,targets = None):
        logits = self.token_embedding_table(inputs)
        if targets is None:
            loss = none
        else :
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    def generation(self,inputs,max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(inputs)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=1)
            inputs_next = torch.multinomial(probs, num_samples = 1)
            inputs = torch.cat((inputs, inputs_next), dim=1)
        return inputs




In [44]:
m = BigramModel(vocab_size)
logits, loss = m(xb,yb)

In [47]:
loss

tensor(4.8786, grad_fn=<NllLossBackward0>)